#### 1. Import pandas library

In [151]:
import pandas as pd

#### 2. Import users table:

In [192]:
df = pd.read_csv("users_table.csv")
df.head(3)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN


#### 3. Rename Id column to userId

In [193]:
df.rename(columns= {"Id": "userId"}, inplace=True)
df.head(3)



,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN


#### 4. Import posts table:

In [194]:
df2 = pd.read_csv("posts_table.csv")
df2.head(3)

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN


#### 5. Rename Id column to postId and OwnerUserId to userId

In [195]:
df2.rename(columns={"Id": "postId", "OwnerUserId": "userId"}, inplace=True)
df2.head(3)

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [196]:
user_columns = df[["userId", "Reputation", "Views", "UpVotes", "DownVotes"]]
print(user_columns.head(3))

post_columns = df2[["postId", "Score", "userId", "ViewCount", "CommentCount"]]
print(post_columns.head(3))

   userId  Reputation  Views  UpVotes  DownVotes
0      -1           1      0     5007       1920
1       2         101     25        3          0
2       3         101     22       19          0
   postId  Score  userId  ViewCount  CommentCount
0       1     23     8.0     1278.0             1
1       2     22    24.0     8198.0             1
2       3     54    18.0     3613.0             4


#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [197]:
df3 = pd.merge(user_columns, post_columns)
df3.head(3)

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [198]:
print(f"Total of missing numbers:", df3.isnull().sum().sum())

null_columns = list(df3.columns[df3.isnull().any()])
null_columns

Total of missing numbers: 23572


['ViewCount']

In [199]:
df5 = df3.fillna(0)
df5.describe()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
count,38962.000000,38962.000000,38962.000000,38962.000000,38962.000000,38962.000000,38962.000000,38962.000000,38962.000000
mean,6079.063087,7281.091679,1400.648016,914.799677,43.841050,22960.799651,4.083081,472.509573,2.014630
std,5224.896435,15164.527714,3423.886887,2296.527060,161.797079,13696.932471,6.561843,2423.961137,2.674018
min,-1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,-19.000000,0.000000,0.000000
25%,1317.000000,147.000000,16.000000,4.000000,0.000000,11325.250000,1.000000,0.000000,0.000000
50%,4856.000000,909.000000,124.000000,65.000000,1.000000,22373.500000,2.000000,0.000000,1.000000
75%,9651.000000,7931.000000,1050.000000,582.000000,16.000000,33688.500000,5.000000,253.000000,3.000000
max,55226.000000,87393.000000,20932.000000,11442.000000,1920.000000,48325.000000,192.000000,175495.000000,45.000000


#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [135]:
print(round(df3["ViewCount"].count() / df3["postId"].count() * 100, 2))

"""Since the column ViewCount has 39.5% of NOT NULLS, 
and because we are dealing with the number os views for specific posts,
in my opinion we should assume that a null == 0 replace null with 0.

Seems dangerous for me to delete the column or even replace the nulls by the average of views, 
because we are can affect negatively further analysis
"""

df4 = df3
df4["ViewCount"] = df4["ViewCount"].fillna((df4["ViewCount"].mean()))
df4.head()
df4.describe()

#df['Item_Weight'] = df['Item_Weight'].fillna((df['Item_Weight'].mean()))

39.5


,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
count,38962.000000,38962.000000,38962.000000,38962.000000,38962.000000,38962.000000,38962.000000,38962.000000,38962.000000
mean,6079.063087,7281.091679,1400.648016,914.799677,43.841050,22960.799651,4.083081,1196.225991,2.014630
std,5224.896435,15164.527714,3423.886887,2296.527060,161.797079,13696.932471,6.561843,2352.363892,2.674018
min,-1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,-19.000000,14.000000,0.000000
25%,1317.000000,147.000000,16.000000,4.000000,0.000000,11325.250000,1.000000,595.250000,0.000000
50%,4856.000000,909.000000,124.000000,65.000000,1.000000,22373.500000,2.000000,1196.225991,1.000000
75%,9651.000000,7931.000000,1050.000000,582.000000,16.000000,33688.500000,5.000000,1196.225991,3.000000
max,55226.000000,87393.000000,20932.000000,11442.000000,1920.000000,48325.000000,192.000000,175495.000000,45.000000


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [204]:
df5["ViewCount"] = df5["ViewCount"].astype('int64')
df5.dtypes

userId          int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
postId          int64
Score           int64
ViewCount       int64
CommentCount    int64
dtype: object